In [2]:
import sys
import logging
import json
import numpy as np
import pandas as pd
from datetime import datetime
import time

In [3]:
MODE = '6and3'
dfknobs = pd.read_json(f'dfknobs_{MODE}.json')

In [10]:
MODES = {'BARE_SH1P12345_SH3P12345_SH4P12345_SL1_SL2_SL3_SVD0':19,# 5+5+5+3+1 = 19
         'BARE_SH1P12345_SH3P12345_SH4_SL1_SL2_SL3_SVD0':15, # 5+5+1+3+1 = 15
         'BARE_SH1P12345_SH3_SH4_SL1_SL2_SL3_SVD0':11, # 5+1+1+3+1 = 11
         'BARE_SH1_SH3_SH4_SL1_SL2_SL3_SVD0':7, # 1+1+1+3+1 = 7
         'DW_SH1P12345_SH3_SH4_SL1_SL2_SL3_SH1DW081828_SH3DW081828_SH4DW081828_SVD0':20, # 5+1+1+1+1+1+3+3+3+1 = 20
         'DW_SH1_SH3_SH4_SL1_SL2_SL3_SH1DW081828_SH3DW081828_SH4DW081828_SVD0':16, # 1+1+1+1+1+1+3+3+3+1 = 16
         }
families = [f'SH{i}' for i in [1,3,4]] + [f'SL{i}' for i in [1,2,3]]

mode_groups = {}
for m,cnt in MODES.items():
    groups = []
    parts = m.split('_')[1:]
    for p in parts:
        if p in families:
            groups.append(p)
        elif p == 'SVD0':
            groups.append(p)
        elif p.endswith('P12345'):
            groups.extend(f"{p.split('P12345')[0]}-P{i}" for i in [1,2,3,4,5])
        elif p.endswith('DW081828'):
            groups.extend(f"{p.split('DW081828')[0]}-DW{i:02d}" for i in [8,18,28])
    
    mode_groups[m] = groups
    assert len(groups) == cnt, f'{m} {groups} {cnt}'
    

In [13]:
dfdict = {}
for m in MODES:
    index = mode_groups[m].copy()
    indexnosvd = index.copy()
    columns = mode_groups[m].copy()
    if 'SVD0' in columns:
        index.remove('SVD0')
        indexnosvd.remove('SVD0')
        index.extend(['SM1A', 'SM1B', 'SM2B'])
    else:
        raise
    df = pd.DataFrame(np.zeros((len(index), len(columns))), index=index, columns=columns)
    df.loc[['SM1A', 'SM1B', 'SM2B'],'SVD0'] = dfknobs.loc[['SM1A', 'SM1B', 'SM2B'],'SVD0']
    for i in indexnosvd:
        df.loc[i,i] = 1.0
    dfdict[m] = df

In [14]:
dfdict['BARE_SH1_SH3_SH4_SL1_SL2_SL3_SVD0']

,SH1,SH3,SH4,SL1,SL2,SL3,SVD0
SH1,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH3,0.0,1.0,0.0,0.0,0.0,0.0,0.000000
SH4,0.0,0.0,1.0,0.0,0.0,0.0,0.000000
SL1,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
SL2,0.0,0.0,0.0,0.0,1.0,0.0,0.000000
SL3,0.0,0.0,0.0,0.0,0.0,1.0,0.000000
SM1A,0.0,0.0,0.0,0.0,0.0,0.0,-0.663666
SM1B,0.0,0.0,0.0,0.0,0.0,0.0,0.744816
SM2B,0.0,0.0,0.0,0.0,0.0,0.0,-0.069260


In [15]:
dfdict['BARE_SH1P12345_SH3_SH4_SL1_SL2_SL3_SVD0']

,SH1-P1,SH1-P2,SH1-P3,SH1-P4,SH1-P5,SH3,SH4,SL1,SL2,SL3,SVD0
SH1-P1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH1-P2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH1-P3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH1-P4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH1-P5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000
SH4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000
SL1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
SL2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000
SL3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000


In [16]:
dump = {m:df.to_json() for m,df in dfdict.items()}

In [17]:
with open(f'dfknobs_all.json', 'w') as f:
    json.dump(dump, f)